In [1]:
import numpy as np
from scipy.integrate import odeint, solve_ivp
from scipy.special import kn
from scipy.optimize import newton, bisect
import matplotlib.pyplot as plt
from astropy import constants as ct
from ipywidgets import interact,fixed,widgets,interactive
import warnings 
  
# Settings the warnings to be ignored 
warnings.filterwarnings('ignore') 

From the Friedmann equation, considering SM radiation domination, the Hubble parameter can be written as:

$$
H(T)=\sqrt{\dfrac{\rho_R(T)}{3M_P^2}}
$$

where $\rho_R(T)$ is given by:

$$
\rho_R(T)=\dfrac{\pi^2}{30}g_\star(T)T^4
$$

with $g_\star=106.75$ being the relativistic degrees of freedom. Setting the chemical potential equal to zero, the DM number density in equilibrium is:

$$n_{eq}(T)=\dfrac{g_{DM}}{2\pi^2}m_{DM}^2TK_2\left(\dfrac{m_{DM}}{T}\right)$$

The Boltzmann equation can be written in terms of the yield $Y$ as:

$$\dfrac{\mathrm{d}Y}{\mathrm{d}x}=-\dfrac{\langle\sigma v\rangle s}{xH}(Y^2-Y_{eq}^2)$$

where $x=m_{DM}/T$, $Y(T)=n(T)/s(T)$, $Y_{eq}=n_{eq}(T)/s(T)$ and $s$ is given by:

$$
s(T)=\dfrac{2\pi^2}{45}g_{\star s}(T)T^3
$$

This equation depicts the process $\mathrm{DM}+\mathrm{DM}\to\mathrm{SM}+\mathrm{SM}$.

The freeze-out time $x_f$ is defined as the $x$ at which $n_{eq}\langle\sigma v\rangle=H$. The expected yield $Y_0$ that is consistent with the observations is given by:

$$
Y_0=\dfrac{3M_P^2\Omega_{DM}h^2(2.13\times10^{-42}\text{ GeV})^2}{ms_0}
$$

with $\Omega_{DM}h^2\approx 0.12$.

In [2]:
gstar = 106.75
MP = 2.4e18 #GeV
T0 = 2.725*ct.k_B.value/ct.e.value*1e-9
abundance = 0.12
gstars0 = 3.91
gstar0 = 3.38

In [3]:
def Yeq(x,mass,g):
    neq = g/(2*np.pi**2)*mass**3/x*kn(2,x)
    s = 2*np.pi**2/45*gstar*(mass/x)**3
    return neq/s

def WIMPs(x,Y,mass,sigma,g):
    H = (np.pi**2*gstar/90)**0.5*(mass/x)**2/MP
    s = 2*np.pi**2/45*gstar*(mass/x)**3
    dYdx = -sigma*s/(x*H)*(Y**2 - Yeq(x,mass,g)**2)
    return dYdx

def xf(mass,sigma,g):
    func = lambda x: g/(2*np.pi**2)*mass**3/x*kn(2,x)*sigma - (np.pi**2*gstar/90)**0.5*(mass/x)**2/MP
    try:
        xf = newton(func,2)
        return xf
    except:
        return np.nan

def Y0m(mass,sigma,g):
    lam = (2*gstars0*np.pi/15)*(10/gstar0)**0.5*mass*sigma*MP
    xfo = xf(mass,sigma,g)
    return (xfo/lam)*mass
    
Y0m = np.vectorize(Y0m)

In [4]:
xini = 0.1
xfin = 1e4
m0 = 100
sigma0 = 7e-10
g0 = 2
Yini = Yeq(xini, m0, g0)
sol = solve_ivp(WIMPs, [xini,xfin], [Yini,], method='BDF', rtol=1e-8, atol=1e-10, args=(m0,sigma0,g0))

In [5]:
def sigma_fit(mass,g):
    Y = 4.3e-10/mass
    func = lambda sigma: 4.3e-10/mass-xf(mass,sigma,g)/(2*gstars0*np.pi/15)*(10/gstar0)**0.5*mass*sigma*MP
    sig = newton(func,1e-9)
    return sig

In [6]:
def graf1(mass,sigma,g):
    xini = 0.1
    xfin = 1e4
    Yini = Yeq(xini,mass,g)
    xss = np.linspace(xini,30,1000)
    sol = solve_ivp(WIMPs, [xini,xfin], [Yini,], method='BDF', rtol=1e-8, atol=1e-10, args=(mass,sigma,g))
    plt.loglog(sol.t,sol.y[0],label=f'Ym={sol.y[0][-1]*mass}')
    plt.loglog(xss,Yeq(xss,mass,g))
    plt.hlines(4.3e-10/mass,xini,xfin,color='k',linestyle=':')
    plt.vlines(xf(mass,sigma,g),2e-8,3e-3,color='r')
    #plt.vlines(xf(mass,sigma),2e-8,3e-3,color='g')
    #plt.vlines(Yxm(mass,sigma,freezeout=True)[1],2e-8,3e-3,color='g',linestyle=':')
    #plt.ylim(2e-8,3e-3)
    plt.grid()
    plt.legend()
    plt.show()
    
slider_mass = widgets.FloatLogSlider(value=1e-3, base=10, min=-3, max=3,
                                  step=0.0001, description='mass')
slider_sigma = widgets.FloatLogSlider(value=1e-12, base=10, min=-15, max=-7,
                                  step=0.1, description='sigma')
slider_g = widgets.FloatSlider(value=2, min=1, max=3,
                                  step=1, description='g')
# Crea la función interactiva

interactive_plot1 = interactive(graf1, mass=slider_mass, sigma=slider_sigma, g=slider_g,
                                continuous_update=0)

# Muestra la función interactiva
interactive_plot1

interactive(children=(FloatLogSlider(value=0.001, description='mass', max=3.0, min=-3.0, step=0.0001), FloatLo…

In [7]:
def WIMPs_SA(x,Y,mass,sigma,g):
    H = (np.pi**2*gstar/90)**0.5*(mass/x)**2/MP
    s = 2*np.pi**2/45*gstar*(mass/x)**3
    dYdx = -sigma*s/(x*H)*(Y**2 - Y*Yeq(x,mass,g))
    return dYdx

def SIMPs(x,Y,mass,sigma,g):
    H = (np.pi**2*gstar/90)**0.5*(mass/x)**2/MP
    s = 2*np.pi**2/45*gstar*(mass/x)**3
    dYdx = -sigma*s**2/(x*H)*(Y**3 - Y**2*Yeq(x,mass,g))
    return dYdx

In [8]:
def graf2(mass,sigma,g):
    xini = 0.1
    xfin = 1e4
    Yini = Yeq(xini,mass,g)
    xss = np.linspace(xini,30,1000)
    sol = solve_ivp(SIMPs, [xini,xfin], [Yini,], method='BDF', rtol=1e-8, atol=1e-10, args=(mass,sigma,g))
    plt.loglog(sol.t,sol.y[0],label=f'Ym={sol.y[0][-1]*mass}')
    plt.loglog(xss,Yeq(xss,mass,g))
    plt.hlines(4.3e-10/mass,xini,xfin,color='k',linestyle=':')
    #plt.vlines(xf(mass,sigma,g),2e-8,3e-3,color='r')
    #plt.vlines(xf(mass,sigma),2e-8,3e-3,color='g')
    #plt.vlines(Yxm(mass,sigma,freezeout=True)[1],2e-8,3e-3,color='g',linestyle=':')
    #plt.ylim(2e-8,3e-3)
    plt.grid()
    plt.legend()
    plt.show()
    
slider_mass = widgets.FloatLogSlider(value=1e-3, base=10, min=-4, max=2,
                                  step=0.0001, description='mass')
slider_sigma = widgets.FloatLogSlider(value=1e-12, base=10, min=-10, max=-3,
                                  step=0.1, description='sigma')
slider_g = widgets.FloatSlider(value=2, min=1, max=3,
                                  step=1, description='g')
# Crea la función interactiva

interactive_plot2 = interactive(graf2, mass=slider_mass, sigma=slider_sigma, g=slider_g,
                                continuous_update=0)

# Muestra la función interactiva
interactive_plot2

interactive(children=(FloatLogSlider(value=0.001, description='mass', max=2.0, min=-4.0, step=0.0001), FloatLo…

In [9]:
def graf3(mass,sigma,g):
    xini = 0.1
    xfin = 1e4
    Yini = Yeq(xini,mass,g)
    xss = np.linspace(xini,30,1000)
    sol = solve_ivp(WIMPs_SA, [xini,xfin], [Yini,], method='BDF', rtol=1e-8, atol=1e-10, args=(mass,sigma,g))
    plt.loglog(sol.t,sol.y[0],label=f'Ym={sol.y[0][-1]*mass}')
    plt.loglog(xss,Yeq(xss,mass,g))
    plt.hlines(4.3e-10/mass,xini,xfin,color='k',linestyle=':')
    #plt.vlines(xf(mass,sigma,g),2e-8,3e-3,color='r')
    #plt.vlines(xf(mass,sigma),2e-8,3e-3,color='g')
    #plt.vlines(Yxm(mass,sigma,freezeout=True)[1],2e-8,3e-3,color='g',linestyle=':')
    #plt.ylim(2e-8,3e-3)
    plt.grid()
    plt.legend()
    plt.show()
    
slider_mass = widgets.FloatLogSlider(value=1e-3, base=10, min=-4, max=2,
                                  step=0.0001, description='mass')
slider_sigma = widgets.FloatLogSlider(value=1e-12, base=10, min=-10, max=-3,
                                  step=0.1, description='sigma')
slider_g = widgets.FloatSlider(value=2, min=1, max=3,
                                  step=1, description='g')
# Crea la función interactiva

interactive_plot3 = interactive(graf3, mass=slider_mass, sigma=slider_sigma, g=slider_g,
                                continuous_update=0)

# Muestra la función interactiva
interactive_plot3

interactive(children=(FloatLogSlider(value=0.001, description='mass', max=2.0, min=-4.0, step=0.0001), FloatLo…